Required Dependencies

In [1]:
import geemap

ModuleNotFoundError: No module named 'geemap'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

import utils

from utils.ground_stations import plot_stations_folium

%matplotlib inline

## Data Extraction
- Accesssing TAHMO Data and CHIRPS Data

###  Extract TAHMO data using filter-stations

Documentation: https://filter-stations.netlify.app/

A config file is provided with the api keys to access TAHMO Data
```json
{
    "apiKey": "",
    "apiSecret": "",}

```

In [2]:
import json
from utils.filter_stations import RetrieveData

# Load the config file
with open('config.json', 'r') as f:
    config = json.load(f)

# Set the api key and secret
api_key = config['apiKey']
api_secret = config['apiSecret']


# Initialize the class
rd = RetrieveData(apiKey=api_key, 
                  apiSecret=api_secret)

East Africa: April-May 2025 floods (Kenya/Rwanda/Uganda), where heavy rains overwhelmed stations.


In [3]:
# get Kenyan TAHMO weather stations
ke_stations = rd.get_stations_info(countrycode='KE')
print(f"Number of stations in Kenya: {len(ke_stations)}")

# Rwandan Stations
rw_stations = rd.get_stations_info(countrycode='RW')
print(f"Number of stations in Rwanda: {len(rw_stations)}")

# Ugandan Stations
ug_stations = rd.get_stations_info(countrycode='UG')
print(f"Number of stations in Uganda: {len(ug_stations)}")

# Concatenate the East African stations
eac_stations = pd.concat([ke_stations, rw_stations, ug_stations])
print(f"Total number of stations in East Africa: {len(eac_stations)}")

Number of stations in Kenya: 180
Number of stations in Rwanda: 19
Number of stations in Uganda: 61
Total number of stations in East Africa: 260


In [4]:
# Plot the stations on a map
plot_stations_folium([ke_stations, rw_stations, ug_stations],
                     colors=["blue", "red", "green"])

In [5]:
# create the data directory/ground for the ground observations
os.makedirs("data/ground", exist_ok=True)

# save this data to be reused later
eac_stations.to_csv("data/ground/eac_stations.csv", index=False)

In [8]:
# Extract the precipitation data for the East African stations (April -May 2025)
start_date = "2025-04-01"
end_date = "2025-05-31"

eac_precip = rd.multiple_measurements(stations_list=eac_stations['code'].tolist(),
                                     startDate=start_date,
                                     endDate=end_date,
                                     variables=['pr'],
                                     csv_file = "data/ground/eac_precip_apr_may_2025.csv")


Retrieving data for stations:   0%|          | 0/260 [00:00<?, ?it/s]

An error occurred: None: Max retries exceeded with url: /services/measurements/v2/stations/TA00026/measurements/controlled?start=2025-04-01T00%3A00%3A00Z&end=2025-05-31T00%3A00%3A00Z&variable=pr (Caused by None)


### Extracting CHIRPS Data for the Eat Africa Region (Kenya, Uganda, Rwanda)